## Загрузим нужные библиотеки

In [1]:
!pip install catboost
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import uniform
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn import svm
from imblearn.over_sampling import SMOTE 
from sklearn.preprocessing import PolynomialFeatures
import tensorflow as tf 
import numpy as np
import pickle
from sklearn.naive_bayes import MultinomialNB
import catboost as cb
%matplotlib inline

2022-11-09 22:03:50.302014: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-09 22:03:50.430599: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-09 22:03:50.430617: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-09 22:03:50.465498: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-09 22:03:51.082407: W tensorflow/stream_executor/platform/de

In [2]:
df = pd.read_csv("train_frame.csv",low_memory=False)
df

,id,oper_type + oper_attr,index_oper,type,priority,is_privatecategory,class,is_in_yandex,is_return,weight,...,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address,label,is_wrong,total_mean
0,6818780,26,17235,18,1,1,0.0,2,0,0.0,...,58950.0,779126.0,8290896.0,0,0,0,0,0,0,0.093974
1,9907176,20,23,4,1,1,0.0,1,0,0.0,...,83318932.0,132175590.0,136819803.0,0,0,0,0,0,0,0.966056
2,3304275,15,16331,19,1,1,0.0,2,0,0.0,...,3233068.0,6479360.0,52708071.0,0,1,0,0,0,1,0.122929
3,9020937,16,6325,19,1,1,0.0,2,0,0.0,...,653280.0,2714208.0,19562334.0,0,0,0,0,0,0,0.138747
4,3082311,17,17315,18,1,1,0.0,2,0,1.0,...,27911.0,344830.0,4719186.0,0,0,0,0,0,0,0.073070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5999995,9958614,19,23,4,1,1,0.0,1,0,0.0,...,116432632.0,180702765.0,188407812.0,0,0,1,0,0,1,0.959104
5999996,2234489,19,4697,19,1,1,0.0,2,0,0.0,...,144063.0,1911433.0,15582018.0,0,0,0,0,0,0,0.122669
5999997,4304572,24,18,19,3,1,0.0,1,0,2.0,...,10648.0,60624000.0,75592387.0,0,0,0,0,0,0,0.801986
5999998,6550634,15,6,19,3,1,0.0,1,0,0.0,...,4972424.0,20063762.0,39988530.0,0,1,0,0,0,1,0.501738


Обьединим список не нужных строк с списком строк типа object

In [3]:
col_obj = df.select_dtypes(include=['object']).columns.values
col_obj = list(set(col_obj) ^ set(["id", "label"]))

In [4]:
col_obj

['id', 'label']

In [5]:
X = df.drop(col_obj, axis = 1)
y = df["label"]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, stratify=y)

In [7]:
sm = SMOTE(random_state=42)

X_train, y_train = sm.fit_resample(X_train, y_train)

In [8]:
X_train.shape

(9909334, 29)

## Обучение модели

In [9]:
scaler = StandardScaler()
#scaler.fit(X)
# logistic = OneVsRestClassifier(RandomForestClassifier(max_depth=22,n_jobs=-1))
#logistic=GaussianNB()
#pipe = Pipeline([('normalizer', scaler), ('classifier', logistic)])
#pipe.fit(X_train, y_train)

clf1=HistGradientBoostingClassifier(learning_rate=0.1, max_depth= 8, min_samples_leaf=100)
clf2=RandomForestClassifier(max_depth=15,n_jobs=9)
eclf1 = VotingClassifier([('gb', clf2),('rf', clf2)], voting='soft',weights=[1,1.5])
pipe = Pipeline([('normalizer', scaler), ('classifier', eclf1)])
pipe.fit(X_train, y_train)

pool_train = cb.Pool(X, y)
params = {                  # 'task_type':'GPU',
                            'iterations':1200,
                            'depth':8,
                            'random_state':2,
                            'learning_rate':0.01,
                            'eval_metric':'Recall',
                            'loss_function':'MultiCrossEntropy'}
clf_with_aux = cb.CatBoostClassifier(**params)
clf_with_aux.fit(pool_train)


predcat = clf_with_aux.predict_proba(X_test)
voting = pipe.predict_proba(X_test)


y_pred = ((voting + predcat) / 2).argmax(axis=1)


print(classification_report(y_test, y_pred))



parametrs = {
              'max_depth': range (1,35),
            }
grid = GridSearchCV(RandomForestClassifier(), parametrs, scoring='roc_auc', cv=2,n_jobs=9) 
grid.fit(X_test, y_test) 
grid.best_params_



pkl_filename = "model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(pipe, file)

with open(pkl_filename, 'rb') as file:
    model = pickle.load(file)

nb_classifier = GaussianNB()

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}
gs_NB = GridSearchCV(estimator=nb_classifier, 
                 param_grid=params_NB, 
                 cv=2,   # use any cross validation technique 
                 verbose=1, 
                 scoring='roc_auc') 
gs_NB.fit(X_train, y_train)

gs_NB.best_params_


gb_grid_params = {'learning_rate': [0.1, 0.05, 0.02, 0.01],
              'max_depth': [4, 6, 8],
              'min_samples_leaf': [20, 50,100,150],
              #'max_features': [1.0, 0.3, 0.1] 
              }
print(gb_grid_params)

gb_gs = HistGradientBoostingClassifier()

clf = GridSearchCV(gb_gs,
                               gb_grid_params,
                               cv=2,
                               scoring='roc_auc',
                               verbose = 3, 
                               n_jobs=2);
clf.fit(X_train, y_train);
clf.best_params_

# Нейронка

In [16]:
from keras import models
from keras import layers
import tensorflow as tf 
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(290,activation='sigmoid',input_shape=(29,)))
    model.add(layers.Dense(15,activation='relu'))
    model.add(layers.Dense(1,activation='sigmoid'))
    model.compile(optimizer='rmsprop',# Вы также можете указать параметры для оптимизатора через optimizer = optimizer.RMSprop (lr = 0.001)
                  loss='binary_crossentropy', # Эквивалент потерь = loss.binary_crossentropy
                  metrics=[tf.keras.metrics.Recall()]) # Эквивалентно метрике = [metircs.binary_accuracy]
    return model
model = build_model()


history = model.fit(X_train, y_train,
                    epochs=2, # Итерировать 20 раз по полному набору данных
                    batch_size=512, # Размер каждой партии 512
                    validation_data=(X_test,y_test))

Epoch 1/2


2022-11-09 22:24:24.646438: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2298965488 exceeds 10% of free system memory.


19355/19355 [==============================] - 55s 3ms/step - loss: 0.4554 - recall_1: 0.9312 - val_loss: 0.5374 - val_recall_1: 0.9389
Epoch 2/2
19355/19355 [==============================] - 54s 3ms/step - loss: 0.4539 - recall_1: 0.9355 - val_loss: 0.5381 - val_recall_1: 0.9437


In [17]:
predictions = model.predict(X_test)
pred=[]
for i in predictions:
    pred.append(round(float(i)))
print('нейронка',classification_report(y_test, pred))

28125/28125 [==============================] - 23s 828us/step
нейронка               precision    recall  f1-score   support

           0       1.00      0.65      0.79    874353
           1       0.07      0.94      0.14     25647

    accuracy                           0.66    900000
   macro avg       0.54      0.80      0.46    900000
weighted avg       0.97      0.66      0.77    900000



## Оценка точности

In [18]:
pred =pipe.predict(X_test)

In [19]:
from sklearn.metrics import recall_score

score = recall_score(y_test, pred, average = "macro" )

In [20]:
print("Recall", score)

Recall 0.965951949644467


In [21]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,  pipe.predict_proba(X_test)[:, 1])

0.9880068054796481

In [22]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98    874353
           1       0.44      0.97      0.61     25647

    accuracy                           0.96    900000
   macro avg       0.72      0.97      0.79    900000
weighted avg       0.98      0.96      0.97    900000



In [23]:
print('Требуемая величина', 0.1*score+0.9*roc_auc_score(y_test,  pipe.predict_proba(X_test)[:, 1]))

Требуемая величина 0.9858013198961301


# Предсказание 

In [24]:
pred_data=pd.read_csv('test_dataset_test.csv',low_memory=False)


In [25]:
df=pred_data

In [26]:
df["priority"] = pd.Categorical(df["priority"])
df["priority"].astype('category').cat.codes
df["priority"] = df["priority"].cat.codes
df["is_in_yandex"] = pd.Categorical(df["is_in_yandex"])
df["is_in_yandex"].astype('category').cat.codes
df["is_in_yandex"] = df["is_in_yandex"].cat.codes
df["is_return"] = pd.Categorical(df["is_return"])
df["is_return"].astype('category').cat.codes
df["is_return"] = df["is_return"].cat.codes
df["oper_type + oper_attr"] = pd.Categorical(df["oper_type + oper_attr"])
df["oper_type + oper_attr"].astype('category').cat.codes
df["oper_type + oper_attr"] = df["oper_type + oper_attr"].cat.codes
df["index_oper"] = pd.Categorical(df["index_oper"])
df["index_oper"].astype('category').cat.codes
df["index_oper"] = df["index_oper"].cat.codes
df["type"] = pd.Categorical(df["type"])
df["type"].astype('category').cat.codes
df["type"] = df["type"].cat.codes
df["is_privatecategory"] = pd.Categorical(df["is_privatecategory"])
df["is_privatecategory"].astype('category').cat.codes
df["is_privatecategory"] = df["is_privatecategory"].cat.codes
df["name_mfi"] = pd.Categorical(df["name_mfi"])
df["name_mfi"].astype('category').cat.codes
df["name_mfi"] = df["name_mfi"].cat.codes
df["weight"] = round(df["weight"]/1000)
df['is_wrong']=df['is_wrong_sndr_name']+df['is_wrong_rcpn_name']+df['is_wrong_phone_number']+df['is_wrong_address']
df['total_mean']=df['total_qty_over_index_and_type']/df['total_qty_over_index']

In [27]:
df = df.fillna(0)


In [28]:
df=df.drop(['id'], axis = 1)

In [29]:
pred_data['label']=pipe.predict(df)

In [30]:
pred_data[['id','label']].to_csv('bestmodel.csv', index=False)

# Нейро 

In [31]:
predictions = model.predict(df)
pred=[]
for i in predictions:
    pred.append(round(float(i)))
pred_data['label']=pred
pred_data[['id','label']].to_csv('bestmodelnero.csv', index=False)

125000/125000 [==============================] - 106s 845us/step
